In [1]:
# Datasets (in general)
# 1. How comprehensive are these datasets?
# 2. How recent are they? 
# 3. Can any aspects be independently validated/verified?
# 4. What makes these datasets authoritative?

# LBSM
# How accurate is the London Building Stock Model?
# What social indicators to use? Why?

# EPC 
# How reliable/variable is the EPC data? 

# Methodology
# Any critical consideration?

In [2]:
# LBSMv2 как источник, в котором данных больше всего. EPC как источник, на который можно опереться в плане качества данных (проверка по пересечению с LBSMv2) + 
# обновление данных на основе EPC (потому что там данные свежее)
# LBSMv2 - продукт Greater London Authority в партнёрстве (в описаниях фигурирует UCL Energy Institute), используемый для городских программ энергоэффективности
# (https://data.london.gov.uk/dataset/london-building-stock-model-lbsm-296oy/) - каких?
# Ограничения:
#   (1) Данные - снэпшот на определенную дату (какую?)

# EPC - государственный реестр энерго-сертификатов с публичным доступом и документацией - кто его делает?. Государственные источники авторитетные. 
# (https://epc.opendatacommunities.org/?utm_source=chatgpt.com)
# Ограничения:
#   (1) Данные - только для части домов (каких?)

# Camden Open Data - государственный источник с открытыми данными по району - а его кто делает?
# (https://opendata.camden.gov.uk/?)

# Верификационные стратегии:
# 1) сравнение данных LBSMv2 и EPC по пересекающимся домам и выяснить:
#   (1) сколько процентов - исходные данные и сколько - смоделированные
#   (2) в каких домах выводы на основе моделирования
# 2) сравнение зон охраны с зонами, указанными в LBSMv2:
#   (1) сколько домов в зонах охраны
#   (2) насколько данные правильные в LBSMv2

# какие именно дома мы не трогаем в conservation area?
# Camden Retrofit at Scale - почитать (https://camden.moderngov.co.uk/mgConvert2PDF.aspx?ID=122232&utm_source=chatgpt.com)

In [3]:
# Тема? Почему именно такая тема? - готово 

# Почему именно такие признаки? - мы здесь - ну мы их написали, но не объяснили

# Почему именно такая методология?
# Почему именно такие данные?

# Действительно ли сложно проводить ретрофитинг в зонах охраны? - готово 

In [4]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 
import numpy as np
import hdbscan

In [5]:
lbsm = pd.read_csv('../data/row/lbsmv2_camden.csv')
lbsm.head(3)

,uprn,os_topo_toid,easting,northing,postcode_locator,administrative_area,oa21cd,lsoa21cd,lsoa21nm,lsoa11cd,...,avg_tilt,imd19_national_decile,imd19_income_decile,loac_supergroup,loac_group,fuel_poverty,heat_risk_quintile,listed_building_grade,conservation_area_flag,conservation_area_site_id
0,200125425,1000004055135,524239.02,185679.53,NW2 2RT,Camden,E00000684,E01000143,Barnet 041D,E01000143,...,35.0,3,2,F - Young Families and Mainstream Employment,F2 - Social Rented Sector and Diverse Origins,11.4,2,Unknown,not in conservation area,not in conservation area
1,200125422,1000004055130,524218.90,185697.60,NW2 2RT,Camden,E00000684,E01000143,Barnet 041D,E01000143,...,35.0,3,2,F - Young Families and Mainstream Employment,F2 - Social Rented Sector and Diverse Origins,11.4,2,Unknown,not in conservation area,not in conservation area
2,200125427,1000004055137,524243.74,185671.32,NW2 2RT,Camden,E00000684,E01000143,Barnet 041D,E01000143,...,35.0,3,2,F - Young Families and Mainstream Employment,F2 - Social Rented Sector and Diverse Origins,11.4,2,Unknown,not in conservation area,not in conservation area


In [6]:
# CURRENT_ENERGY_RATING, POTENTIAL_ENERGY_RATING - буква, например, 'C'
# CURRENT_ENERGY_EFFICIENCY, POTENTIAL_ENERGY_EFFICIENCY - число от 1 до 100

# CO2_EMISSIONS_CURRENT, CO2_EMISSIONS_POTENTIAL - прикольно, что они есть
# UPRN - уникальный идентификатор здания, можно скрестить с LBSM

epc = pd.read_csv('../data/row/epc_camden/certificates.csv', usecols=['UPRN', 'CURRENT_ENERGY_RATING', 'POTENTIAL_ENERGY_RATING',
                                                                       'CURRENT_ENERGY_EFFICIENCY', 'POTENTIAL_ENERGY_EFFICIENCY'])
epc.head(3)

,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,UPRN
0,C,C,69,77,5086054.0
1,C,C,74,80,5143505.0
2,D,B,64,81,5198075.0


In [7]:
# retrofit candidates: non-conservation areas with low EPC ratings
low_epc = ['D', 'E', 'F', 'G', 'F-G']

lbsm_cand = lbsm[
    (lbsm['conservation_area_flag'] == 'not in conservation area') &
    (lbsm['epc_rating'].isin(low_epc))
].copy()

# uprn are located into different buildings - topo
print(f'URPN num: {lbsm_cand["uprn"].nunique()}, Topo TOID num: {lbsm_cand["os_topo_toid"].nunique()}')

bldg_pts = (
    lbsm_cand
    .groupby('os_topo_toid', as_index=True)[['easting', 'northing']]
    .median()
)

X = bldg_pts[['easting', 'northing']].to_numpy()

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=25,
    min_samples=10,
    metric='euclidean',
    cluster_selection_method='eom'
)

labels = clusterer.fit_predict(X)
bldg_pts['cluster_id'] = labels

# cluster stats
unique_clusters = sorted(set(labels) - {-1})
print("Clusters from Topo TOID:", len(unique_clusters))
print("Noise share:", (labels == -1).mean()) # 16% домохозяйств вне кластеров

# export clustered buildings
bldg_pts = gpd.GeoDataFrame(
    bldg_pts,
    geometry=gpd.points_from_xy(bldg_pts['easting'], bldg_pts['northing']),
    crs="EPSG:27700"
)

bldg_pts.reset_index().to_file('clustered_buildings.geojson', driver='GeoJSON')

URPN num: 21206, Topo TOID num: 7102
Clusters from Topo TOID: 45
Noise share: 0.15784286116586876


In [132]:
bldg_pts['cluster_id'].value_counts()

cluster_id
 22    1230
-1     1121
 36     547
 28     482
 7      368
 39     282
 27     225
 41     183
 2      177
 18     165
 40     164
 44     157
 8      154
 38     117
 4      116
 11     112
 20     109
 34     106
 31      91
 33      86
 24      78
 37      74
 3       71
 43      71
 13      55
 14      52
 16      50
 9       48
 29      42
 10      41
 35      41
 5       41
 1       39
 30      39
 19      39
 26      39
 15      39
 25      33
 21      31
 23      28
 12      28
 42      27
 32      27
 17      26
 0       26
 6       25
Name: count, dtype: int64

In [126]:
lbsm_cand.shape

(21206, 63)

In [8]:
lbsm_clustered = lbsm_cand.merge(
    bldg_pts[['cluster_id']],
    how='left',
    left_on='os_topo_toid',
    right_on='os_topo_toid'
)

In [ ]:
lbsm_archetypes = lbsm_clustered.copy()

# Fabric profiling
lbsm_archetypes["wall_need_solid"]  = (lbsm_archetypes["wall_type"] == "solid") & (lbsm_archetypes["wall_insulation"] == "uninsulated")
lbsm_archetypes["wall_need_cavity"] = (lbsm_archetypes["wall_type"] == "cavity") & (lbsm_archetypes["wall_insulation"] == "uninsulated")

lbsm_archetypes["roof_need_pitched"] = (lbsm_archetypes["roof_type"] == "pitched") & (lbsm_archetypes["roof_insulation"] == "uninsulated")
lbsm_archetypes["roof_need_flat"]    = (lbsm_archetypes["roof_type"] == "flat") & (lbsm_archetypes["roof_insulation"] == "uninsulated")

lbsm_archetypes["glazing_need_single"]    = (lbsm_archetypes["glazing_type"] == "single/partial")
lbsm_archetypes["glazing_need_secondary"] = (lbsm_archetypes["glazing_type"] == "secondary")

# Heating profiling
lbsm_archetypes["heating_system"] = lbsm_archetypes["main_heat_type"].astype(str) + " | " + lbsm_archetypes["main_fuel_type"].astype(str)

def heat_bucket(row):
    fuel = row["main_fuel_type"]
    ht = str(row["main_heat_type"]).lower()

    if fuel == "mains gas":
        return "Gas-based heating"
    if fuel == "electricity":
        if "storage" in ht:
            return "Electric storage/resistance"
        return "Electric heating"
    return "Other / unknown"

lbsm_archetypes["heating_bucket"] = lbsm_archetypes.apply(heat_bucket, axis=1)

# EPC gap profiling
lbsm_archetypes["efficiency_gap"] = lbsm_archetypes["potential_epc_score"] - lbsm_archetypes["epc_score"]
lbsm_archetypes["gap_band"] = pd.cut(lbsm_archetypes["efficiency_gap"], bins=[-1, 4, 9, 100], labels=["Low", "Medium", "High"])

# measures bundling
def measure_bundle(r):
    measures = []
    if r["wall_need_solid"]:  measures.append("Solid wall insulation")
    if r["wall_need_cavity"]: measures.append("Cavity wall insulation")
    if r["roof_need_pitched"]: measures.append("Loft/roof insulation (pitched)")
    if r["roof_need_flat"]:    measures.append("Roof insulation (flat)")
    if r["glazing_need_single"]: measures.append("Glazing upgrade (single/partial)")
    if r["glazing_need_secondary"]: measures.append("Improve/replace secondary glazing")
    if r["heating_bucket"] in ["Gas-based heating", "Electric storage/resistance"]:
        measures.append(f"Heating pathway: {r['heating_bucket']}")
    return measures

In [98]:
cluster_profile = (
    lbsm_archetypes[lbsm_archetypes["cluster_id"] >= 0]
    .groupby("cluster_id")
    .agg(
        n=("uprn", "nunique"),
        share_solid_wall=("wall_need_solid", "mean"),
        share_cavity_wall=("wall_need_cavity", "mean"),
        share_roof_pitched=("roof_need_pitched", "mean"),
        share_roof_flat=("roof_need_flat", "mean"),
        share_glazing_single=("glazing_need_single", "mean"),
        share_glazing_secondary=("glazing_need_secondary", "mean"),
        median_gap=("efficiency_gap", "median"),
        share_gap_high=("gap_band", lambda x: (x == "High").mean()),
        top_heating=("heating_bucket", lambda x: x.value_counts().index[0]),
        top_heating_share=("heating_bucket", lambda x: x.value_counts(normalize=True).iloc[0]),
    )
    .reset_index()
)

In [99]:
# сколько домохозяйств в кластере страдает от проблемы
cluster_profile.head(3)

,cluster_id,n,share_solid_wall,share_cavity_wall,share_roof_pitched,share_roof_flat,share_glazing_single,share_glazing_secondary,median_gap,share_gap_high,top_heating,top_heating_share
0,0,72,0.388889,0.111111,0.027778,0.069444,0.500000,0.041667,10.0,0.541667,Gas-based heating,0.736111
1,1,216,0.129630,0.328704,0.046296,0.125000,0.013889,0.000000,6.0,0.370370,Gas-based heating,0.833333
2,2,291,0.024055,0.034364,0.006873,0.061856,0.474227,0.000000,6.0,0.347079,Gas-based heating,0.993127


In [100]:
cp = cluster_profile.copy()

THRESH = 0.5

measure_cols = {
    "Solid wall insulation (uninsulated solid walls)": "share_solid_wall",
    "Cavity wall insulation (uninsulated cavity walls)": "share_cavity_wall",
    "Loft/roof insulation (pitched, uninsulated)": "share_roof_pitched",
    "Roof insulation (flat, uninsulated)": "share_roof_flat",
    "Glazing upgrade (single/partial)": "share_glazing_single",
    "Glazing improvements (secondary glazing)": "share_glazing_secondary",
}

def build_bundle(row, thresh=THRESH):
    items = [name for name, col in measure_cols.items() if row[col] >= thresh]
    return items

cp["bundle_list"] = cp.apply(build_bundle, axis=1)
cp["bundle"] = cp["bundle_list"].apply(lambda xs: "; ".join(xs) if xs else "Mixed / no dominant bundle")

cp['bundle'].value_counts()

bundle
Solid wall insulation (uninsulated solid walls)                                      26
Mixed / no dominant bundle                                                            9
Solid wall insulation (uninsulated solid walls); Glazing upgrade (single/partial)     5
Glazing upgrade (single/partial)                                                      4
Cavity wall insulation (uninsulated cavity walls)                                     1
Name: count, dtype: int64

In [101]:
cp.head(3)

,cluster_id,n,share_solid_wall,share_cavity_wall,share_roof_pitched,share_roof_flat,share_glazing_single,share_glazing_secondary,median_gap,share_gap_high,top_heating,top_heating_share,bundle_list,bundle
0,0,72,0.388889,0.111111,0.027778,0.069444,0.500000,0.041667,10.0,0.541667,Gas-based heating,0.736111,[Glazing upgrade (single/partial)],Glazing upgrade (single/partial)
1,1,216,0.129630,0.328704,0.046296,0.125000,0.013889,0.000000,6.0,0.370370,Gas-based heating,0.833333,[],Mixed / no dominant bundle
2,2,291,0.024055,0.034364,0.006873,0.061856,0.474227,0.000000,6.0,0.347079,Gas-based heating,0.993127,[],Mixed / no dominant bundle


In [102]:
# Качественный показатель: профиль мер (из bundle) - 5 штук - What?
# Социально-экономический показатель: насколько эти меры нужны населению? - Who?
# Насколько радмкальные возможны улучшения энергоэффективности? - How much?

In [103]:
soc = (
    lbsm_archetypes[lbsm_archetypes["cluster_id"] >= 0]
    .groupby("cluster_id")
    .agg(
        imd_income_median=("imd19_income_decile", "median"),
        fuel_poverty_median=("fuel_poverty", "median"),
    )
    .reset_index()
)
cp = cp.merge(soc, on="cluster_id", how="left")

cp["imd_norm"] = (10 - cp["imd_income_median"]) / 9  # 1 is worst, 0 is best
cp["fuel_pov_norm"] = cp["fuel_poverty_median"].rank(pct=True)  # 1 is worst, 0 is best

cp["social_index"] = 0.5*cp["imd_norm"] + 0.5*cp["fuel_pov_norm"]
# cp['social_index_category'] = cp.apply(lambda row: 'High' if row['social_index'] > 0.5 else 'Low', axis=1)

In [104]:
lbsm_archetypes['efficiency_gap'] = lbsm_archetypes['potential_epc_score'] - lbsm_archetypes['epc_score']

eff_gap = (
    lbsm_archetypes[lbsm_archetypes["cluster_id"] >= 0]
    .groupby("cluster_id")
    .agg(
        efficiency_gap_median=("efficiency_gap", "median"),
    )
    .reset_index()
)

cp = cp.merge(eff_gap, on="cluster_id", how="left")
cp["efficiency_index"] = cp["efficiency_gap_median"].rank(pct=True)  # 1 is worst, 0 is best
# cp['efficiency_index_category'] = cp.apply(lambda row: 'High' if row['efficiency_index'] > 0.5 else 'Low', axis=1)

In [105]:
cp_join = cp[['cluster_id', 'bundle', 'social_index', 'efficiency_gap_median', 'efficiency_index', 'imd_income_median', 'fuel_poverty_median']].copy()

bldg_pts_enriched = (
    bldg_pts
    .reset_index()  
    .merge(cp_join, on='cluster_id', how='left')
)

In [106]:
bldg_pts_enriched.to_file('clustered_buildings_enriched.geojson', driver='GeoJSON')

In [107]:
gdf = bldg_pts_enriched.copy()

if not isinstance(gdf, gpd.GeoDataFrame):
    gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:27700")

gdf0 = gdf[gdf["cluster_id"] >= 0].copy()

cluster_hulls = (
    gdf0.dissolve(by="cluster_id")
        .convex_hull
        .reset_index()
)

cluster_hulls = gpd.GeoDataFrame(cluster_hulls, geometry=0, crs=gdf0.crs).rename(columns={0: "geometry"})
cluster_hulls.head()

,cluster_id,geometry
0,0,"POLYGON ((531096 181642, 531084 181805, 531099..."
1,1,"POLYGON ((530332 182517, 530079 182740, 530491..."
2,2,"POLYGON ((529893 184157, 529806 184159, 529780..."
3,3,"POLYGON ((529325 182441, 529240 182473, 529117..."
4,4,"POLYGON ((528995 182421, 528894 182440, 528890..."


In [108]:
cluster_hulls_enriched = (
    cluster_hulls
    .reset_index()  
    .merge(cp_join, on='cluster_id', how='left')
)

In [109]:
cluster_hulls_enriched = gpd.GeoDataFrame(cluster_hulls_enriched, geometry='geometry', crs=gdf0.crs)
cluster_hulls_enriched['geometry'] = cluster_hulls_enriched.buffer(30)
cluster_hulls_enriched = gpd.GeoDataFrame(cluster_hulls_enriched, geometry='geometry', crs=gdf0.crs)
cluster_hulls_enriched.to_file('cluster_hulls_enriched.geojson', driver='GeoJSON')

In [110]:
n_by_cluster = (
    lbsm_archetypes
    .loc[lbsm_archetypes["cluster_id"] >= 0]
    .groupby("cluster_id")["uprn"]
    .nunique()
    .reset_index(name="n_households")
)

cluster_hulls_enriched = cluster_hulls_enriched.merge(
    n_by_cluster,
    on="cluster_id",
    how="left"
)

cluster_hulls_enriched["n_households"] = cluster_hulls_enriched["n_households"].fillna(0).astype(int)

cluster_hulls_enriched.head()

,index,cluster_id,geometry,bundle,social_index,efficiency_gap_median,efficiency_index,imd_income_median,fuel_poverty_median,n_households
0,0,0,"POLYGON ((531119.008 181622.748, 531116.997 18...",Glazing upgrade (single/partial),0.316667,10.0,0.666667,7.0,8.9,72
1,1,1,"POLYGON ((530334.249 182487.084, 530331.26 182...",Mixed / no dominant bundle,0.716667,6.0,0.211111,2.0,10.5,216
2,2,2,"POLYGON ((529894.41 184127.033, 529892.311 184...",Mixed / no dominant bundle,0.883333,6.0,0.211111,2.0,12.1,291
3,3,3,"POLYGON ((529338.972 182414.452, 529336.423 18...",Solid wall insulation (uninsulated solid walls),0.777778,2.0,0.044444,3.0,11.7,254
4,4,4,"POLYGON ((529015.235 182398.851, 529012.852 18...",Mixed / no dominant bundle,0.811111,9.0,0.555556,3.0,12.0,244


In [111]:
def bundle_to_class(b):
    if pd.isna(b):
        return None
    if "Mixed" in b:
        return "Mixed"
    if "Solid wall" in b:
        return "Solid wall"
    if "Cavity wall" in b:
        return "Cavity wall"
    if "Glazing" in b:
        return "Glazing"
    if "Roof" in b or "Loft" in b:
        return "Roof"
    return "Other"

cluster_bundle = cluster_hulls_enriched[['cluster_id','bundle']].copy()
cluster_bundle['bundle_class'] = cluster_bundle['bundle'].apply(bundle_to_class)

df = lbsm_archetypes.drop(columns=['bundle']).copy()

df = df.merge(cluster_bundle[['cluster_id','bundle','bundle_class']], on='cluster_id', how='left')

# helper flags
glazing_need = df['glazing_need_single'] | df['glazing_need_secondary']
roof_need = df['roof_need_pitched'] | df['roof_need_flat']

# 1) помечаем кластеры с multi-measure bundle (в строке есть ';')
df['bundle_is_multi'] = df['bundle'].astype(str).str.contains(';', regex=False)

# 2) считаем bundle_match как обычно
df['bundle_match'] = False
df.loc[df['bundle_class'] == 'Solid wall', 'bundle_match'] = df['wall_need_solid']
df.loc[df['bundle_class'] == 'Cavity wall', 'bundle_match'] = df['wall_need_cavity']
df.loc[df['bundle_class'] == 'Glazing', 'bundle_match'] = glazing_need
df.loc[df['bundle_class'] == 'Roof', 'bundle_match'] = roof_need

# 3) ключевой шаг: отключаем подсветку для multi-measure кластеров
df.loc[df['bundle_is_multi'], 'bundle_match'] = False

# 4) и для mixed тоже
df.loc[df['bundle_class'] == 'Mixed', 'bundle_match'] = False
df.loc[df['cluster_id'] == -1, 'bundle_match'] = False

df['geometry'] = df.apply(lambda row: Point(row.easting, row.northing), axis=1)
df_filtered = df[df['cluster_id'] != -1]
gdf = gpd.GeoDataFrame(df_filtered, geometry='geometry', crs='EPSG:27700')
gdf.to_crs(epsg=4326).to_file('clustered_buildings_bundle_match.geojson', driver='GeoJSON')

In [112]:
single = cluster_hulls_enriched['bundle'].isin(['Glazing upgrade (single/partial)',
                                       'Solid wall insulation (uninsulated solid walls)',
                                       'Cavity wall insulation (uninsulated cavity walls)'])

clusters_single = cluster_hulls_enriched[single]

cluster_hulls_enriched[single].sort_values(['social_index', 'efficiency_gap_median', 'n_households'], ascending=[False, False, False])

,index,cluster_id,geometry,bundle,social_index,efficiency_gap_median,efficiency_index,imd_income_median,fuel_poverty_median,n_households
37,37,37,"POLYGON ((528502.783 184317.516, 528503.99 184...",Glazing upgrade (single/partial),0.938889,9.0,0.555556,1.0,12.1,276
16,16,16,"POLYGON ((529385.194 183028.167, 529382.458 18...",Solid wall insulation (uninsulated solid walls),0.922222,4.0,0.133333,2.0,13.0,349
19,19,19,"POLYGON ((529219.646 183678.088, 529218.659 18...",Solid wall insulation (uninsulated solid walls),0.883333,9.0,0.555556,3.0,15.3,92
30,30,30,"POLYGON ((524801.437 184471.588, 524799.021 18...",Solid wall insulation (uninsulated solid walls),0.850000,11.0,0.744444,3.0,12.3,93
44,44,44,"POLYGON ((525036.265 184600.466, 525033.086 18...",Solid wall insulation (uninsulated solid walls),0.850000,7.0,0.333333,3.0,12.3,311
13,13,13,"POLYGON ((529903.821 182711.774, 529902.554 18...",Solid wall insulation (uninsulated solid walls),0.811111,6.0,0.211111,2.0,11.6,178
3,3,3,"POLYGON ((529338.972 182414.452, 529336.423 18...",Solid wall insulation (uninsulated solid walls),0.777778,2.0,0.044444,3.0,11.7,254
21,21,21,"POLYGON ((524295.069 185509.701, 524292.851 18...",Cavity wall insulation (uninsulated cavity walls),0.777778,23.0,1.000000,2.0,11.4,32
15,15,15,"POLYGON ((529570.107 182938.628, 529567.08 182...",Glazing upgrade (single/partial),0.755556,3.0,0.088889,2.0,10.9,162
31,31,31,"POLYGON ((528954.432 184234.087, 528952.753 18...",Solid wall insulation (uninsulated solid walls),0.722222,7.0,0.333333,4.0,11.7,339


In [117]:
gdf.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich